In [1]:
import pandas as pd
import numpy as np

from env import get_db_url

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

import warnings
warnings.filterwarnings("ignore")

In [2]:
url = get_db_url('zillow')
query = '''
            
SELECT bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, yearbuilt, taxamount, fips
FROM properties_2017

LEFT JOIN propertylandusetype USING(propertylandusetypeid)

WHERE propertylandusedesc IN ("Single Family Residential",                       
                              "Inferred Single Family Residential")'''

data = pd.read_sql(query, url)
df = data.copy() #use copy to explore on
df.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
0,0.0,0.0,NaN,27516.0,NaN,NaN,6037.0
1,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
2,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
3,0.0,0.0,NaN,2108.0,NaN,174.21,6037.0
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0


In [3]:
features_dictionary = pd.read_excel('zillow_data_dictionary.xlsx')
features_dictionary

,Feature,Description
0,'airconditioningtypeid',Type of cooling system present in the home (i...
1,'architecturalstyletypeid',"Architectural style of the home (i.e. ranch, ..."
2,'basementsqft',Finished living area below or partially below...
3,'bathroomcnt',Number of bathrooms in home including fractio...
4,'bedroomcnt',Number of bedrooms in home
5,'buildingqualitytypeid',Overall assessment of condition of the buildi...
6,'buildingclasstypeid',"The building framing type (steel frame, wood f..."
7,'calculatedbathnbr',Number of bathrooms in home including fractio...
8,'decktypeid',Type of deck (if any) present on parcel
9,'threequarterbathnbr',Number of 3/4 bathrooms in house (shower + si...


#### Rename columns

In [4]:
df = df.rename(columns = {'bedroomcnt':'bedrooms', 
                          'bathroomcnt':'bathrooms', 
                          'calculatedfinishedsquarefeet':'area',
                          'taxvaluedollarcnt':'tax_value', 
                          'yearbuilt':'year_built',
                          'taxamount': 'tax_amount'})

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2152864 entries, 0 to 2152863
Data columns (total 7 columns):
 #   Column      Dtype  
---  ------      -----  
 0   bedrooms    float64
 1   bathrooms   float64
 2   area        float64
 3   tax_value   float64
 4   year_built  float64
 5   tax_amount  float64
 6   fips        float64
dtypes: float64(7)
memory usage: 115.0 MB


In [6]:
df.shape

(2152864, 7)

In [7]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
bedrooms,2152853.0,3.287195,0.954757,0.00,3.0000,3.000,4.0000,25.00
bathrooms,2152853.0,2.230687,0.999281,0.00,2.0000,2.000,3.0000,32.00
area,2144379.0,1862.855178,1222.125124,1.00,1257.0000,1623.000,2208.0000,952576.00
tax_value,2152371.0,461896.052361,699675.940049,1.00,188170.0000,327671.000,534527.0000,98428909.00
year_built,2143526.0,1960.949681,22.162196,1801.00,1949.0000,1958.000,1976.0000,2016.00
tax_amount,2148422.0,5634.863752,8178.908996,1.85,2534.9725,4108.945,6414.3175,1337755.86
fips,2152864.0,6048.377330,20.433289,6037.00,6037.0000,6037.000,6059.0000,6111.00


In [8]:
#null values in each row?

df.isnull().sum()

bedrooms        11
bathrooms       11
area          8485
tax_value      493
year_built    9338
tax_amount    4442
fips             0
dtype: int64

WHERE predictions_2017 transactiondate LIKE '2017%'

In [9]:
#project MVP req's 

url2 = get_db_url('zillow')
query2 = '''
            
SELECT bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, yearbuilt, fips, transactiondate
FROM properties_2017

RIGHT JOIN propertylandusetype USING(propertylandusetypeid)
LEFT JOIN predictions_2017 USING (id)

WHERE propertylandusedesc IN ("Single Family Residential",                       
                              "Inferred Single Family Residential")
 '''

data2 = pd.read_sql(query2, url2)
df2 = data2.copy()
df2.head()

OperationalError: (pymysql.err.OperationalError) (2003, "Can't connect to MySQL server on '157.230.209.171' (timed out)")
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [ ]:
df2.shape

In [ ]:
df2.isnull().sum()

In [ ]:
df2.transactiondate.describe()

In [ ]:
#project MVP req's #inner join acceptable bc looking to subset data w/ transactions in 2017

url3 = get_db_url('zillow')
query3 = '''
            
SELECT bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, yearbuilt, fips, transactiondate
FROM properties_2017

LEFT JOIN propertylandusetype USING(propertylandusetypeid)
JOIN predictions_2017 USING (id)

WHERE propertylandusedesc IN ("Single Family Residential",                       
                              "Inferred Single Family Residential")                        
'''

data3 = pd.read_sql(query3, url3)
df3 = data3.copy()
df3.head()

In [ ]:
df3.shape

In [ ]:
df3.isnull().sum()

In [ ]:
df3.transactiondate.sort_values(ascending = False) 
#shows there is one value from 2018-05-25, need to delete

In [ ]:
#reassign df w/o 2018 transactiondate data
df3 = df3[(df3.transactiondate <= '2017-12-31')]

In [ ]:
df3.transactiondate.sort_values(ascending = False)

In [ ]:
#what is the percentage that would be left if null values dropped?

round(df3.dropna().shape[0] / df3.shape[0], 4)

# 99.52 % of data retained, can drop nulls

In [ ]:
#acquire data

def acquire_zillow(use_cache = True):
    if os.path.exists('zillow.csv') and use_cache:
        print('Using cached csv')
        return pd.read_csv('zillow.csv')
    print('Acquring from SQL database')
    url = get_db_url('zillow')
    query = '''
            
    SELECT bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, yearbuilt, fips, transactiondate
    FROM properties_2017

    LEFT JOIN propertylandusetype USING(propertylandusetypeid)
    JOIN predictions_2017 USING (id)

    WHERE propertylandusedesc IN ("Single Family Residential",                       
                              "Inferred Single Family Residential")'''
    
    #create df
    df = pd.read_sql(query, url)

    #rename df columns
    df = df.rename(columns = {'bedroomcnt': 'bedrooms',
                              'bathroomcnt': 'bathrooms',
                              'calculatedfinishedsquarefeet': 'area',
                              'taxvaluedollarcnt': 'tax_value',
                              'yearbuilt': 'year_built'})
    
    df.to_csv('zillow.csv', index = False)                          
    return df

